In [ ]:
import requests
import json
import time
from tqdm.auto import tqdm
import random

f = open('secret.json')
j = json.load(f)

api_key = j['API_KEY']
api_tok = j['API_TOKEN']

print(f'API KEY  : {api_key}')
print(f'API TOKEN: {api_tok}')

In [ ]:
url_prefix = 'https://api.themoviedb.org/3/movie/'

headers = {
    'user-agent' : 
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'
}

# https://image.tmdb.org/t/p/original/[poster_path]

poster_prefix = 'https://image.tmdb.org/t/p/original'

#### Get all movies 

```
model Movie {
  id         String    @id @default(cuid())
  title      String
  overview   String
  poster_url String
  crew       Person[]
  genres     Genre[]
  keywords   Keyword[]
  release_date String
  countries    Country[]
  companies    Company[]
  budget       Int
  revenue      Int
  popularity   Float
  vote_average Float
  vote_count   Int
  ratings      Rating[]
}
```

In [ ]:
movies = []

persons = [ {} ]
users = [ {} ]
genres = [ {}]
ratings = [ {} ]
companies = [ {} ]
countries = [ {} ]
keywords = [ {} ]
languages = [ {} ]

In [ ]:
def getData(url):
    file_path = url.split('/3/')[1]
    
    file_path = file_path.replace('/', '_')
    file_path = file_path.replace('?', '_')
    
    try:
        f = open(f'./raw_data/{file_path}')
        json_dat = json.load(f)
        f.close()
#         print(f'Load: ./raw_data/{file_path}')
        return json_dat
    except:
        res = requests.get(url, headers = headers)
        json_dat = json.loads(res.text)
        f = open(f'./raw_data/{file_path}', 'w')
        f.write(json.dumps(json_dat))
        f.close()
#         print(f'New file: ./raw_data/{file_path}')
    return json_dat

In [ ]:
import pandas as pd

links = pd.read_csv('links.csv', index_col=0, na_filter=False)

links.head()

In [ ]:
genre_cnt = 1
all_genre = {}

def addGenre(now):
    global genre_cnt, all_genre, genres
    
    if not (now['id'] in all_genre):
        all_genre[now['id']] = genre_cnt
        
        g = {}
        g['id'] = now['id']
        g['name'] = now['name'] or ""
        g['movies'] = []
        
        genres.append(g)
        
        genre_cnt += 1
        
    return all_genre[now['id']]

In [ ]:
rating_cnt = 1

def download_rating(idx, m):
    global ratings, rating_cnt, users
    
    url = url_prefix + str(idx) + f'/reviews?api_key={ api_key }'
    
    json_dat = getData(url)
    
    pages = json_dat['total_pages']
    
    for i in range(1, pages+1):
        url_now = url + f'&page={i}'
        
        json_dat = getData(url_now)
        
        for k in json_dat['results']:
            user_detail = k['author_details']
            
            if user_detail['rating'] == None: continue
            
            now_data = {}
            now_data['id'] = rating_cnt
            now_data['rating'] = user_detail['rating']
            now_data['movieId'] = idx
            
            user_id = getUserID(user_detail['username'])
            now_data['userId'] = user_id
            
            now_data['createdAt'] = k['created_at'] or ""
            now_data['updatedAt'] = k['updated_at'] or ""
            
            users[user_id]['ratings'].append(rating_cnt)
            
            m['ratings'].append(rating_cnt)
            
            rating_cnt += 1
            ratings.append(now_data)
            
    return m

In [ ]:
keyw_cnt = 1
all_keyword = { }

def download_keyword(idx, m):
    global keywords, keyw_cnt, all_keyword

    url = url_prefix + str(idx) + f'/keywords?api_key={ api_key }'
    json_dat = getData(url)

    for i in json_dat['keywords']:
        if not(i['id'] in all_keyword):
            all_keyword[i['id']] = keyw_cnt

            k = {}
            k['id'] = i['id']
            k['name'] = i['name'] or ""
            k['movies'] = []

            keywords.append(k)
            keyw_cnt += 1

        poi = all_keyword[i['id']]
        keywords[poi]['movies'].append(idx)
        
        m['keywords'].append(all_keyword[i['id']])
            
    return m

In [ ]:
person_cnt = 1
all_person = { }

def download_credits(idx, m):
    global person_cnt, all_person, persons
    
    url = url_prefix + str(idx) + f'/credits?api_key={ api_key }'
    json_dat = getData(url)
    
    for i in json_dat['cast']:
        u = { }
        if not(i['id'] in all_person):
            all_person[i['id']] = person_cnt
            
            u = { }
            u['id'] = i['id']
            u['name'] = i['name'] or ""
            u['gender'] = i['gender'] or 0
            u['popularity'] = i['popularity'] or 0.0
            u['movies'] = []
            
            persons.append(u)
            person_cnt += 1
        
        poi = all_person[i['id']]
        persons[poi]['movies'].append(idx)
        
        m['crew'].append(all_person[i['id']])
    
    return m

# download_credits(862)

In [ ]:
user_cnt = 1
all_username = { }

def getUserID(username):
    global user_cnt, all_username, users

    if not (username in all_username):
        all_username[username] = user_cnt
        
        now = {}
        now['id'] = user_cnt
        now['ratings'] = []
        users.append(now)
        
        user_cnt += 1
        
    return all_username[username]

In [ ]:
comp_cnt = 1
all_comp = { }

def addCompany(now):
    global comp_cnt, all_comp, companies
    
    if not (now['id'] in all_comp):
        all_comp[now['id']] = comp_cnt
        
        c = {}
        c['id'] = now['id']
        c['name'] = now['name'] or ""
        c['movies'] = []
        
        companies.append(c)
        
        comp_cnt += 1
        
    return all_comp[now['id']]

In [ ]:
coty_cnt = 1
all_country = { }

def addCountry(now):
    global all_country, coty_cnt, countries
    
    iso = now['iso_3166_1']
    if not(iso in all_country):
        all_country[iso] = coty_cnt
        
        c = {}
        c['id'] = coty_cnt
        c['iso_3166_1'] = now['iso_3166_1']
        c['name'] = now['name'] or ""
        c['movies'] = []
        
        countries.append(c)
        coty_cnt += 1
    
    return all_country[iso]

In [ ]:
lang_cnt = 1
all_langs = {}

def addLanguage(now):
    global languages, lang_cnt, all_langs
    
    iso = now['iso_639_1']
    
    if not(iso in all_langs):
        all_langs[iso] = lang_cnt
        
        l = {}
        l['id'] = lang_cnt
        l['iso_639_1'] = now['iso_639_1']
        l['name'] = now['name'] or ""
        l['movies'] = []
        
        lang_cnt += 1
        languages.append(l)
    
    return all_langs[iso]

In [ ]:
def add_movie(d):
    m = {}
    idx = d['id']
    
    m['id'] = idx
    m['title'] = d['original_title'] or ""
    m['overview'] = d['overview'] or ""
    m['runtime'] = d['runtime'] or 0
    if d['poster_path'] == None:
        m['poster_url'] = poster_prefix
    else:
        m['poster_url'] = poster_prefix + d['poster_path']
        
    m['release_date'] = d['release_date'] or ""
    m['budget'] = d['budget'] or 0
    m['revenue'] = d['revenue'] or 0

    m['keywords'] = []
    m['crew'] = []
    m['ratings'] = []
    
    m['vote_count'] = d['vote_count'] or 0
    m['vote_average'] = d['vote_average'] or 0.0
    m['popularity'] = d['popularity'] or 0.0
    
    # genres
    m['genres'] = []
    for i in d['genres']:
        poi = addGenre(i)
        genres[poi]['movies'].append(idx)
        m['genres'].append(all_genre[i['id']])

    # companies
    m['companies'] = []
    for i in d['production_companies']:
        poi = addCompany(i)
        companies[poi]['movies'].append(idx)
        m['companies'].append(all_comp[i['id']])

    # country
    m['countries'] = []
    for i in d['production_countries']:
        poi = addCountry(i)
        iso = i['iso_3166_1']
        countries[poi]['movies'].append(idx)
        m['countries'].append(all_country[iso])

    # languages
    m['spoken_languages'] = []
    for i in d['spoken_languages']:
        poi = addLanguage(i)
        iso = i['iso_639_1']
        languages[poi]['movies'].append(idx)
        m['spoken_languages'].append(all_langs[iso])
    
    m['original_language_id'] = all_langs[d['original_language']]
    
    return m

In [ ]:
def add_movies(idx):
    url = url_prefix + str(idx) + f'?api_key={ api_key }'
    
    json_dat = getData(url)
    
    try:
        if json_dat['success'] == False:
            print(f'Not found on {idx}, url = {url}', end = ' ')
            print(json_dat['status_message'])
            return
    except:
        pass
    
    try:
        m = add_movie(json_dat)

        m = download_keyword(idx, m)   
        m = download_rating(idx, m)
        m = download_credits(idx, m)

        movies.append(m)

    except Exception as e:
        print(f'Error on {idx}, url = {url}')
        print(e, end = "\n\n")
        

### Running

In [ ]:
for i in tqdm(links['tmdbId'].tolist()):
    if i == '': continue 
    add_movies(int(i))

#     break
#     time.sleep(0.1 + random.uniform(0.2, 0.4))

In [ ]:
def demo_output(print_all=False):
    print('ratings', rating_cnt)
    if print_all:
        for i in ratings:
            print(i)

    print('\nusers', user_cnt)
    if print_all:
        for i in users:
            print(i)

    print('\ngenres', genre_cnt)
    if print_all:
        for i in genres:
            print(i)

    print('\ncompany', comp_cnt)
    if print_all:
        for i in companies:
            print(i)

    print('\nkeyword', keyw_cnt)
    if print_all:
        for i in keywords:
            print(i)

    print('\npersons', person_cnt)
    if print_all:
        for i in persons:
            print(i)

    print('\ncountries', coty_cnt)
    if print_all:
        for i in countries:
            print(i)

    print('\nlanguages', lang_cnt)
    if print_all:
        for i in languages:
            print(i)
    
    print('\n\nMovies', len(movies))
    if print_all:
        for i in movies:
            print(i)
    
demo_output()

In [ ]:
# TODO
# remove ratings.rating with null

new_rating = []
null_cnt = 0

for i in ratings:
    if i['rating'] == None:
        null_cnt += 1
        continue
    new_rating.append(i)

print("With null: ", null_cnt)
    
print("New ratings:", len(new_rating))

In [ ]:
root_path = './dataset/'

def output_file(name, data):
    f = open(f'{root_path}{name}.json', 'w')
    f.write('[')
    for i in range(len(data)):
        if i != 0: 
            f.write(',')
        f.write(json.dumps(data[i]))
    f.write(']')
    f.close()

In [ ]:
output_file('movie', movies)
output_file('person', persons)

output_file('country', countries)
output_file('keyword', keywords)
output_file('company', companies)
output_file('genre', genres)

output_file('language', languages)
output_file('rating', new_rating)
output_file('user', users)

---

### Movie Detail Example
```json
{
  "adult": false,
  "backdrop_path": "/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg",
  "belongs_to_collection": {
    "id": 10194,
    "name": "Toy Story Collection",
    "poster_path": "/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg",
    "backdrop_path": "/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg"
  },
  "budget": 30000000,
  "genres": [
    {
      "id": 16,
      "name": "Animation"
    },
    {
      "id": 12,
      "name": "Adventure"
    },
    {
      "id": 10751,
      "name": "Family"
    },
    {
      "id": 35,
      "name": "Comedy"
    }
  ],
  "homepage": "http://toystory.disney.com/toy-story",
  "id": 862,
  "imdb_id": "tt0114709",
  "original_language": "en",
  "original_title": "Toy Story",
  "overview": "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
  "popularity": 89.92,
  "poster_path": "/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg",
  "production_companies": [
    {
      "id": 3,
      "logo_path": "/1TjvGVDMYsj6JBxOAkUHpPEwLf7.png",
      "name": "Pixar",
      "origin_country": "US"
    }
  ],
  "production_countries": [
    {
      "iso_3166_1": "US",
      "name": "United States of America"
    }
  ],
  "release_date": "1995-10-30",
  "revenue": 373554033,
  "runtime": 81,
  "spoken_languages": [
    {
      "english_name": "English",
      "iso_639_1": "en",
      "name": "English"
    }
  ],
  "status": "Released",
  "tagline": "",
  "title": "Toy Story",
  "video": false,
  "vote_average": 7.965,
  "vote_count": 16092
}
```